In [1]:
import gym
import time
from math import sin, cos, radians, log10
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
from cartpoles import Cart, DCMotor, Pole
from colors import Colors
from cartpolesenv import CartPolesEnv

In [2]:
cart = Cart(0.5, 0.05, 0, -0.8, 0.8, Colors.red,
    DCMotor(-12, 12, 0.05, 0.5, 0.05, 0.01, 0.05, Colors.black),
    Pole(0.2, radians(10), 0.2, 0.005, Colors.green, None 
    )
)

dt = 0.01
g = 9.81
env = CartPolesEnv(cart, dt, g)
env.observation_space.shape

c:\Python310\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


(4,)

In [3]:
n_bins = 6
est = KBinsDiscretizer(n_bins=n_bins, encode="ordinal", strategy="uniform")

def discretizer(data, low: list[float], high: list[float]) -> tuple[int,...]:
    est.fit([low, high])
    return tuple(map(int, est.transform([data])[0]))

def obs_discretizer(obs):
    return discretizer(obs, env.observation_space.low, env.observation_space.high)

def action_discretizer(action):
    return discretizer(action, env.action_space.low, env.action_space.high)

def undiscretizer(data: int, low: list[float], high: list[float]):
    est.fit([low, high])
    return est.inverse_transform([data])[0]

def action_undiscretizer(action):
    return undiscretizer(action, env.action_space.low, env.action_space.high)


In [4]:
Q_table = np.zeros(tuple(np.repeat(np.array([n_bins]), env.observation_space.shape[0])) + tuple(np.repeat(np.array([n_bins]), env.action_space.shape[0])), dtype=int)
Q_table.shape

(6, 6, 6, 6, 6)

In [5]:
def policy(state: tuple):
    return (np.argmax(Q_table[state]),)

In [6]:
def new_Q_value(reward: float, new_state: tuple, discount_factor=1.0) -> float:
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

In [7]:
def learning_rate(n: int, min_rate=0.01) -> float:
    return max(min_rate, min(1.0, 1.0 - log10((n+1)/25)))

In [8]:
def exploration_rate(n: int, min_rate=0.1) -> float:
    return max(min_rate, min(1, 1.0-log10((n+1)/25)))

In [9]:
n_episodes = 1000

f = IntProgress(min=0, max=n_episodes) # instantiate the bar
display(f) # display the bar

env_e = env
for e in range(n_episodes):
    f.value += 1

    obs, info = env_e.reset()
    current_state, done = obs_discretizer(obs), False

    while not done:
        action = policy(current_state)
        # print("policy action!", action)

        if np.random.random() < exploration_rate(e):
            action = action_discretizer(env_e.action_space.sample())
            # print("random!")

        # print("action", action)
        # print("undiscretized", action_undiscretizer(action))

        obs, reward, done, info, _ = env_e.step(*action_undiscretizer(action))
        new_state = obs_discretizer(obs)

        lr = learning_rate(e)
        learnt_value = new_Q_value(reward, new_state, 1)
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value

        current_state = new_state
        
        if e == n_episodes-1:
            env.render()

IntProgress(value=0, max=1000)

In [19]:
Q_table

array([[[[[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0]],

         [[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0]],

         [[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0]],

         [[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0]],

         [[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  